# 프롬프트 엔지니어링 기법
https://www.promptingguide.ai/kr/techniques

In [1]:
from dotenv import load_dotenv
import os

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

# 환경 변수 가져오기
API_KEY = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI(api_key=API_KEY)

### Zero-shot Prompting
: 예시가 주어지지 않음

In [2]:
prompt='''
다음 영어 문장을 불어로 번역해줘 :
'Hello, how are you?'
'''
completion = client.chat.completions.create(
    model="gpt-4o-mini", 
    temperature=0.2,
    messages=[
        {"role": "user", 
         "content": prompt}
    ]
)

print(completion.choices[0].message.content)

'Hello, how are you?'는 프랑스어로 'Bonjour, comment ça va ?'입니다.


### Few-shot Prompting 
: 몇 개의 예시를 제공 , 한 개의 예시가 주어지는 경우 --> One-shot

In [3]:
prompt = """
English: 'Hello, how are you?'
French: 'Bonjour, comment ça va?'

English: 'What is your name?'
French: 'Comment vous appelez-vous?'

English: 'Hello,Miss'
French: 
"""

completion = client.chat.completions.create(
    model="gpt-4o-mini", 
    temperature=0.2,
    messages=[
        {"role": "user", 
         "content": prompt}
    ]
)

print(completion.choices[0].message.content)
# 지시가 없어도 예시만 보고 불어로 번역해준다.

French: 'Bonjour, mademoiselle'


In [5]:
prompt = """
아래 예시를 참조해서 알맞은 답변해줘

질문: 이 영화 너무 재미 없어!!'
답변: 부정 평가

질문: 이 영화 돈이 안 아까와..'
답변: 긍정 평가

질문: 시간만 버렸어~'
답변: 부정 평가

질문: 내내 하품만 나옴'
답변: 

질문: 핵 꿀잼~'
답변: 
"""

completion = client.chat.completions.create(
    model="gpt-4o-mini", 
    temperature=0.2,
    messages=[
        {"role": "user", 
         "content": prompt}
    ]
)

print(completion.choices[0].message.content)

질문: 내내 하품만 나옴  
답변: 부정 평가  

질문: 핵 꿀잼~  
답변: 긍정 평가  


### CoT(Chain-of-Thought) Prompting

In [7]:
prompt ="""
철수에게는 사과가 10개 있었습니다 그저께 부터 오늘까지 매일 아침에 2개씩 먹었습니다
오늘 점심에 어머니께서 사과 5개를 더 사다 주셨습니다 오늘 저녁에 친구에게 2개를 주었습니다
철수는 총 몇 개의 사과를 가지고 있을까요?
"""

completion = client.chat.completions.create(
    model="gpt-4o-mini", 
    temperature=0,
    messages=[
        {"role": "user", 
         "content": prompt}
    ]
)

print(completion.choices[0].message.content)  #  7개 : 정답

철수의 사과 개수를 계산해 보겠습니다.

1. 처음에 철수는 사과가 10개 있었습니다.
2. 그저께부터 오늘까지 매일 아침에 2개씩 먹었습니다. 오늘은 3일째이므로:
   - 2개 × 3일 = 6개를 먹었습니다.
3. 따라서 오늘 아침까지 남은 사과는:
   - 10개 - 6개 = 4개입니다.
4. 오늘 점심에 어머니께서 사과 5개를 더 사다 주셨으므로:
   - 4개 + 5개 = 9개입니다.
5. 오늘 저녁에 친구에게 2개를 주었으므로:
   - 9개 - 2개 = 7개입니다.

결론적으로, 철수는 총 7개의 사과를 가지고 있습니다.


In [9]:
prompt ="""
철수에게는 사과가 10개 있었습니다 그저께 부터 오늘까지 매일 아침에 2개씩 먹었습니다
오늘 점심에 어머니께서 사과 5개를 더 사다 주셨습니다 오늘 저녁에 친구에게 2개를 주었습니다
철수는 총 몇 개의 사과를 가지고 있을까요?
"""

completion = client.chat.completions.create(
    model="gpt-3.5-turbo", 
    temperature=0,
    messages=[
        {"role": "user", 
         "content": prompt}
    ]
)

print(completion.choices[0].message.content)  #  11개 or 9개 : 틀린 답변

철수는 현재 11개의 사과를 가지고 있습니다. 처음에 10개의 사과를 가졌고, 매일 아침 2개씩 먹어서 6개를 먹었으며, 어머니께서 5개를 더 사다 주었기 때문에 11개가 되었습니다. 이후 친구에게 2개를 주었기 때문에 현재 11개의 사과를 가지고 있습니다.


In [12]:
prompt ="""
철수에게는 사과가 10개 있었습니다 그저께 부터 오늘까지 매일 아침에 2개씩 먹었습니다
오늘 점심에 어머니께서 사과 5개를 더 사다 주셨습니다 오늘 저녁에 친구에게 2개를 주었습니다
철수는 총 몇 개의 사과를 가지고 있을까요? 단계별로 분리해서 생각해 보세요
"""

completion = client.chat.completions.create(
    model="gpt-3.5-turbo", 
    temperature=0,
    messages=[
        {"role": "user", 
         "content": prompt}
    ]
)

print(completion.choices[0].message.content)  #   9개 : 틀린 답변

1. 처음에 가지고 있던 사과의 개수: 10개
2. 매일 아침에 먹은 사과의 개수: 2개 * 2일 = 4개
3. 어머니가 사다 준 사과의 개수: 5개
4. 친구에게 준 사과의 개수: 2개

따라서, 철수가 현재 가지고 있는 사과의 개수는
10개 (처음에 가지고 있던 개수) - 4개 (아침에 먹은 개수) + 5개 (어머니가 사다 준 개수) - 2개 (친구에게 준 개수) = 9개

따라서, 철수는 현재 9개의 사과를 가지고 있습니다.


In [14]:
prompt="""
1. 철수에게는 사과가 10개 있었습니다.
2. 그저께부터 오늘까지 매일 아침에 2개씩 먹었습니다.
   (3일 동안 2개씩 먹어서 2 * 3 = 6개의 사과를 먹었습니다.)
3. 오늘 점심에 어머니께서 사과 5개를 더 사다 주셨습니다.
4. 오늘 저녁에 친구에게 2개를 주었습니다.
철수는 총 몇 개의 사과를 가지고 있을까요?
"""

completion = client.chat.completions.create(
    model="gpt-3.5-turbo", 
    temperature=0,
    messages=[
        {"role": "user", 
         "content": prompt}
    ]
)

print(completion.choices[0].message.content)  # 7개 : 정답

철수는 현재 7개의 사과를 가지고 있습니다. 처음에 10개의 사과가 있었고, 6개를 먹었으며, 어머니께서 5개를 더 사다 주셨고, 친구에게 2개를 주었기 때문에 10 - 6 + 5 - 2 = 7개의 사과가 남았습니다.


### SC(Self-Consistency) Prompting
: 어러번 질의해서 가장 빈도가 높은 답을 선택한다

In [15]:
from collections import Counter

# 동일한 질문에 대해 여러 번의 답변을 생성하여 Self-Consistency 유도
def generate_responses(prompt, n=5):
    responses = []
    for _ in range(n):
        response =  client.chat.completions.create(
            # model="gpt-4o-mini",
            model="gpt-3.5-turbo",
            messages=[ {"role": "user",  "content": prompt}],
            temperature=0.7,  # 다양성 확보를 위해 온도를 높게 설정
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        responses.append(response.choices[0].message.content)
    return responses

# 가장 자주 등장하는 답변을 선택
# response_count.most_common() 메서드는 Counter 객체에 저장된 요소들의 
# 빈도수를 기준으로 정렬된 리스트를 반환,  [0][0]는 첫번째 요소 선택
def select_most_consistent_response(responses):
    response_count = Counter(responses)
    most_common_response = response_count.most_common(1)[0][0]
    return most_common_response

In [16]:
# 문제 정의
prompt = """
철수에게는 사과가 10개 있었습니다. 그저께부터 오늘까지 매일 아침에 2개씩 먹었습니다.
오늘 점심에 어머니께서 사과 5개를 더 사다 주셨습니다.
오늘 저녁에 친구에게 2개를 주었습니다.
철수는 총 몇 개의 사과를 가지고 있을까요?
"""

# 여러 번 답변 생성
responses = generate_responses(prompt, n=5)

# 가장 일관성 있는 답변 선택
final_answer = select_most_consistent_response(responses)

# 결과 출력
print("Generated Responses:")
for i, response in enumerate(responses, 1):
    print(f"Response {i}: {response}")

print("\nMost Consistent Answer:")
print(final_answer)

# model="gpt-3.5-turbo" 경우에는 9개
# model="gpt-4o-mini" 경우에는 7개

Generated Responses:
Response 1: 철수가 가진 사과의 총 개수는 10개에서 매일 아침 2개씩 먹은 4일을 빼면 2개가 남습니다. 어머니가 사다주신 5개를 더하면 7개가 되고, 친구에게 준 2개를 빼면 최종적으로 철수는 5개의 사과를 가지고 있습니다.
Response 2: 철수는 처음에 10개의 사과를 가졌으며, 매일 아침에 2개씩 먹었으므로 2일 동안 4개를 먹었습니다.
따라서 오늘 점심에 어머니께서 사과 5개를 더 사다 주었으므로 10 - 4 + 5 = 11개의 사과가 있습니다.
그리고 오늘 저녁에 친구에게 2개를 주었으므로 11 - 2 = 9개의 사과를 가지고 있습니다. 따라서 철수는 총 9개의 사과를 가지고 있습니다.
Response 3: 철수는 처음에 10개의 사과를 가지고 있었고, 매일 아침에 2개씩 먹었으므로 2일 동안 총 4개를 먹었습니다. 따라서 10 - 4 = 6개의 사과가 남았습니다.
어머니께서 사과 5개를 더 사다 주었으므로 6 + 5 = 11개의 사과가 있었습니다.
친구에게 2개를 주었으므로 11 - 2 = 9개의 사과가 남았습니다.
따라서 철수는 총 9개의 사과를 가지고 있습니다.
Response 4: 철수가 가지고 있는 사과의 개수는 11개입니다.
철수가 가지고 있던 10개의 사과에서 매일 아침 2개씩 먹어서 2일간 총 4개를 먹었고, 어머니가 사다 주신 5개를 더하면 11개가 됩니다. 이후에 친구에게 2개를 주었기 때문에, 철수가 현재 가지고 있는 사과의 개수는 11개입니다.
Response 5: 철수는 처음에 10개의 사과를 가지고 있었고, 매일 아침에 2개씩 먹었으므로 2일 동안 4개를 먹었습니다. 따라서 10 - 4 = 6개의 사과가 남았습니다.
어머니께서 사과 5개를 더 사다 주었으므로 6 + 5 = 11개의 사과가 되었습니다.
친구에게 2개를 주었으므로 11 - 2 = 9개의 사과를 가지고 있습니다.

따라서 철수는 총 9개의 사과를 가지고 있습니다.

Most Consistent Answer:
철수

In [18]:
from collections import defaultdict
import re
prompt = """
4명의 생존자가 강을 건너야 합니다. 각 생존자가 강을 건너는 데 걸리는 시간은 각각 다릅니다: 3분, 4분, 7분, 12분.
이들은 한 번에 최대 두 명씩만 함께 배를 타고 강을 건널 수 있으며, 배에는 반드시 등불을 지참해야 합니다. 
그러나 등불은 하나뿐이어서, 생존자들이 강을 건널 때마다 등불을 반드시 함께 가져가야 합니다. 
모든 생존자가 가장 효율적으로 강을 건너려면 어떻게 해야 할까요? 
마지막으로 총 소요 시간을 '정답: <정답>분' 형식으로 답변하세요.
"""
def get_most_frequent_answer(prompt, iterations=10):
    answers = defaultdict(int)
    for idx in range(iterations):
        context = [{"role": "user", "content":prompt}]
        response = client.chat.completions.create(
            # model="gpt-3.5-turbo",   # 다양한 답이 생성, 정답을 못 맞춤
            model="gpt-4o-mini",   # 정답이 거의 같음 : 27분, 빈도표: [('27분', 7), ('43분', 1), ('39분', 1), ('24분', 1)]
            messages=context,
            temperature=0.2
        )
        response_content = response.choices[0].message.content
        
        print(f"\n{idx+1}번째 샘플:")
        print(response_content)
         
        match = re.search(r"정답: (\d+분)", response_content)
        
        if match:
            parsed_answer = match.group(1)
            answers[parsed_answer] += 1
        print('-'*60)  
        
    # 빈도가 가장 높은 답을 선택    
    sorted_answers = sorted(answers.items(), key=lambda x: x[1], reverse=True)
    print(f"\n빈도표: {sorted_answers}")
    most_frequent_answer = sorted_answers[0]
    return most_frequent_answer

most_frequent_answer = get_most_frequent_answer(prompt)
print("최빈값:", most_frequent_answer)


1번째 샘플:
이 문제를 해결하기 위해서는 생존자들이 강을 건너는 시간을 최소화하는 전략을 세워야 합니다. 각 생존자의 강을 건너는 시간은 다음과 같습니다:

- A: 3분
- B: 4분
- C: 7분
- D: 12분

효율적인 방법은 다음과 같습니다:

1. A(3분)와 B(4분)가 먼저 강을 건넙니다. (4분 소요)
2. A(3분)가 돌아옵니다. (3분 소요)
3. C(7분)와 D(12분)가 강을 건넙니다. (12분 소요)
4. B(4분)가 돌아옵니다. (4분 소요)
5. A(3분)와 B(4분)가 다시 강을 건넙니다. (4분 소요)

이 과정을 통해 총 소요 시간은 다음과 같습니다:

- 4분 (A와 B가 건너기)
- 3분 (A가 돌아오기)
- 12분 (C와 D가 건너기)
- 4분 (B가 돌아오기)
- 4분 (A와 B가 다시 건너기)

총 소요 시간 = 4 + 3 + 12 + 4 + 4 = 27분

따라서, 정답은 다음과 같습니다:

정답: 27분
------------------------------------------------------------

2번째 샘플:
이 문제를 해결하기 위해서는 생존자들이 강을 건너는 순서를 잘 계획해야 합니다. 각 생존자의 건너는 시간은 다음과 같습니다:

- A: 3분
- B: 4분
- C: 7분
- D: 12분

가장 효율적인 방법은 다음과 같습니다:

1. A와 B가 함께 강을 건넌다. (4분 소요)
   - A와 B가 강을 건너고, B가 등불을 가지고 돌아온다. (4분 + 4분 = 8분)
   
2. C와 D가 함께 강을 건넌다. (12분 소요)
   - C와 D가 강을 건너고, A가 등불을 가지고 돌아온다. (12분 + 3분 = 15분)
   
3. A와 B가 다시 함께 강을 건넌다. (4분 소요)
   - A와 B가 강을 건너고, B가 등불을 가지고 돌아온다. (4분 + 4분 = 19분)
   
4. C와 D가 다시 함께 강을 건넌다. (12분 소요)
   - C와 D가 강을 건너고, A가 등불을 가지고 

#### <br>
## Prompt Chaining (프롬프트 연쇄)
: 복잡한 문제를 해결하기 위해 여러 단계를 거쳐 연속적으로 프롬프트를 작성하는 방법

https://devocean.sk.com/blog/techBoardDetail.do?ID=166026&boardType=techBlog

In [19]:
def ask_openai(prompt, max_tokens=150):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        max_tokens=max_tokens,
        messages=[
                {"role": "user", 
                 "content": prompt}
                ]
    )
    return response.choices[0].message.content

def product_development_chain():
    """제품 개발 체이닝"""
    
    # 1단계 : 제품 아이디어 구체화
    product_idea = "스마트 홈 보안 시스템"
    prompt_1 = f"{product_idea}의 핵심 기능과 그 기능들이 어떻게 사용자에게 가치를 제공할 수 있는지 설명해줘. 이 제품이 기존 시장에서 어떤 문제를 해결하는지에 대해 강조해줘."
    product_summary = ask_openai(prompt_1)
    print('-'*50)
    print('[1단계] : 제품 아이디어 구체화')
    print(f"제품 아이디어: {product_idea}\n제품 요약: {product_summary}\n")
    
    # 2단계 : 고객 요구사항 분석
    prompt_2 = f"{product_summary}을 바탕으로, 이 제품의 주요 타겟 고객이 어떤 요구사항을 가지고 있는지 분석해줘. 이 고객들이 제품을 사용하면서 겪을 수 있는 주요 문제와 기대되는 기능을 설명해줘."
    customer_requirements = ask_openai(prompt_2)
    print('-'*50)
    print('[2단계] : 고객 요구사항 분석')
    print(f"제품 요약: {product_summary}\n\n고객 요구사항 분석:\n{customer_requirements}\n")
    
    # 3단계 : 성공적인 출시 전략
    prompt_3 = f"{customer_requirements}을 기반으로, 이 제품을 성공적으로 출시하기 위한 전략을 제안해줘. 특히, 마케팅 전략과 고객 피드백을 반영하는 방법에 대해 자세히 설명해줘."
    launch_strategy = ask_openai(prompt_3)
    print('-'*50)
    print('[3단계] : 성공적인 출시 전략')
    print(f"고객 요구사항 분석:\n{customer_requirements}\n\n출시 전략:\n{launch_strategy}\n")

if __name__ == "__main__":
    product_development_chain()

--------------------------------------------------
[1단계] : 제품 아이디어 구체화
제품 아이디어: 스마트 홈 보안 시스템
제품 요약: 스마트 홈 보안 시스템은 현대 주택에서 안전과 편리함을 제공하는 중요한 기술로 자리 잡았습니다. 이 시스템의 핵심 기능과 그 기능들이 사용자에게 제공하는 가치는 다음과 같습니다:

### 1. **모니터링 및 경고**
   - **기능**: 실시간 비디오 모니터링, 움직임 감지 센서, 경보 시스템.
   - **가치 제공**: 사용자는 스마트폰이나 태블릿을 통해 언제 어디서나 자택의 상황을 확인할 수 있습니다. 특히, 외부에서 실시간으로 경고를 받은 경우 즉각적인 대응이 가능하여 안전성을 높입니다.

### 2. **원격

--------------------------------------------------
[2단계] : 고객 요구사항 분석
제품 요약: 스마트 홈 보안 시스템은 현대 주택에서 안전과 편리함을 제공하는 중요한 기술로 자리 잡았습니다. 이 시스템의 핵심 기능과 그 기능들이 사용자에게 제공하는 가치는 다음과 같습니다:

### 1. **모니터링 및 경고**
   - **기능**: 실시간 비디오 모니터링, 움직임 감지 센서, 경보 시스템.
   - **가치 제공**: 사용자는 스마트폰이나 태블릿을 통해 언제 어디서나 자택의 상황을 확인할 수 있습니다. 특히, 외부에서 실시간으로 경고를 받은 경우 즉각적인 대응이 가능하여 안전성을 높입니다.

### 2. **원격

고객 요구사항 분석:
스마트 홈 보안 시스템의 주요 타겟 고객은 다음과 같은 특성과 요구사항을 가진 집주인들로 구성됩니다:

### 주요 타겟 고객
1. **바쁜 직장인**: 일상적으로 업무에 바빠 집에 돌아갈 시간이 제한적인 고객.
2. **가족을 둔 부모**: 자녀의 안전을 최우선으로 생각하며 집을 지키고 싶어하는 고객.
3. **고령자 및 장애인**: 자신의 안전과 주거 공간에 대한 모니터링이 필요한 분들

## Tree of Thoughts (ToT)
: **Tree of Thoughts (ToT)**는 대규모 언어 모델(LLM)이 복잡한 문제를 해결하기 위해 다양한 아이디어를 생성하고 평가하는 과정을 시뮬레이션하는 새로운 방법입니다. 마치 사람이 문제에 대한 해결책을 찾기 위해 마음속으로 여러 가지 가능성을 떠올리고 그중 가장 적절한 것을 선택하는 것과 유사합니다. <br>
### ToT가 작동하는 방식
생각의 나무(Tree of Thoughts): 문제를 해결하기 위한 다양한 아이디어와 접근 방식을 가지(branches)로 확장시키는 나무 구조로 표현합니다. 각 가지는 다양한 사고의 흐름을 나타냅니다.

탐색 과정: 나무의 뿌리(root)에서 시작하여 각 가지를 따라가며 가능한 해결 방법들을 평가하고, 최적의 해결책을 찾는 과정을 탐색이라고 합니다. 이 과정에서 여러 가지 경로를 비교하고, 실패하거나 비효율적인 경로는 가지치기(pruning)하여 제거합니다.

분기와 평가: 문제를 해결하는 다양한 경로를 탐색하는 중간 중간에 각 경로가 얼마나 유망한지 평가하고, 유망한 경로를 더 깊이 탐색하며 문제 해결에 접근합니다.]

![image](https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2FTOT.3b13bc5e.png&w=1200&q=75)

In [20]:
def generate_thoughts(prompt, n=3):
    """
    GPT를 사용하여 주어진 프롬프트에 대해 여러 가지 생각(아이디어)을 생성하는 함수.
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        n=n,  # 생성할 생각(아이디어)의 수
        max_tokens=100  # 각 응답의 최대 길이
    )
    thoughts =  [choice.message.content for choice in response.choices]
    return thoughts

# 함수는 초기 생각들을 생성한 후, 각 생각에 대해 더 깊이 탐색하는 과정을 거칩니다.
# 더 깊이 탐색한 결과 중 길이가 가장 긴 응답을 최적의 생각으로 선택
def tree_of_thoughts(prompt, depth=2):
    """
    Tree of Thoughts 방식을 사용하여 최적의 생각을 찾는 함수.
    """
    initial_thoughts = generate_thoughts(prompt)
    print(f"Initial thoughts:\n{initial_thoughts}\n")

    best_thought = None
    for i, thought in enumerate(initial_thoughts):
        print(f"Exploring thought {i + 1}: {thought}")
        next_prompt = f"{thought} 다음에 어떻게 할 수 있을까요?"
        deeper_thoughts = generate_thoughts(next_prompt)

        # 간단한 평가 방법: 길이가 가장 긴 응답을 최선으로 선택
        best_deeper_thought = max(deeper_thoughts, key=len)
        print(f"Deeper thought: {best_deeper_thought}\n")

        # 최적의 생각 갱신
        if not best_thought or len(best_deeper_thought) > len(best_thought):
            best_thought = best_deeper_thought

    print(f"Best thought: {best_thought}")

# 사용 예제
user_prompt = "자연을 보존하기 위한 창의적인 아이디어가 무엇일까요?"
tree_of_thoughts(user_prompt)

Initial thoughts:
['자연을 보존하기 위한 창의적인 아이디어는 여러 가지가 있습니다. 아래에 몇 가지를 소개합니다:\n\n1. **도시 농업**: 도시 내 버려진 공간이나 옥상, 발코니를 활용해 커뮤니티 농장을 만들어 사람들에게 신선한 농산물을 제공하고, 식물들이 대기질을 개선하게 합니다.\n\n2. **플라스틱 대체품 개발**: 생분해성 소재나 천', '자연을 보존하기 위한 창의적인 아이디어는 여러 가지가 있습니다. 다음은 몇 가지 제안입니다.\n\n1. **커뮤니티 정원 만들기**: 지역 주민들이 함께 참여하여 정원을 조성하고 관리하는 프로그램을 운영합니다. 이를 통해 지역 식물과 생태계를 보호하고, 주민들 간의 유대감을 강화할 수 있습니다.\n\n2. **업사이클링 프로젝트**: 일상생활에서 발생하는 쓰레기', '자연을 보존하기 위한 창의적인 아이디어는 다양합니다. 아래 몇 가지 제안을 드리겠습니다.\n\n1. **도시 농업 증가**: 도심 내에서 수직 농업이나 옥상 정원을 활성화하여 식량 생산과 자연 공간을 늘리는 아이디어입니다. 이를 통해 식물들이 자라는 공간을 마련하고, 지역 사회의 식량 자급률을 높일 수 있습니다.\n\n2. **재활용 예술 프로젝트']

Exploring thought 1: 자연을 보존하기 위한 창의적인 아이디어는 여러 가지가 있습니다. 아래에 몇 가지를 소개합니다:

1. **도시 농업**: 도시 내 버려진 공간이나 옥상, 발코니를 활용해 커뮤니티 농장을 만들어 사람들에게 신선한 농산물을 제공하고, 식물들이 대기질을 개선하게 합니다.

2. **플라스틱 대체품 개발**: 생분해성 소재나 천
Deeper thought: 3. **순환 경제 시스템 구축**: 제품의 수명 주기를 늘리고 자원을 효율적으로 사용하기 위해 리사이클링, 재사용, 수리 프로그램을 활성화하여 소비자들이 제품을 엔드오브라이프(EOL) 이후에 적절히 처리할 수 있도록 합니다.

4. **친환경 에너지 솔루션**: 지역 내 태양열 패널, 풍력 터빈 등